<img width="10%" alt="Naas" src="https://landen.imgix.net/jtci2pxwjczr/assets/5ice39g4.png?w=160"/>

# Jupyter Notebooks - Add tags to cells
<a href="https://app.naas.ai/user-redirect/naas/downloader?url=https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/template.ipynb" target="_parent">
<img src="https://img.shields.io/badge/-Open%20in%20Naas-success?labelColor=000000&logo="/>
</a>

**Tags:** #jupyter #awesome-notebooks #tags #snippet

**Author:** [Florent Ravenel](https://www.linkedin.com/in/florent-ravenel/)

## Input

### Import library

In [19]:
import os
import json
from naas_drivers import notion
import pandas as pd

### Variables

In [17]:
# Input
notion_database = 'https://www.notion.so/naas-official/b75fca2e4704433a8fc914032e089054?v=376dd88315454e89bf128433e7ae0420'
NOTION_TOKEN = "secret_HChj6JHeePegALRldeGOwLsoImmHphmlxlA973DrMye"

## Model

### Get Notion DB

In [22]:
def get_notion_df(notion_database):
    database_id = notion_database.split("/")[-1].split("?v=")[0]
    pages = notion.connect(NOTION_TOKEN).database.query(database_id, query={})
    print("📊 Pages in Notion DB:", len(pages))

    columns = pages[0].df().Name.tolist()
    df = pd.DataFrame()

    for page in pages:
        values = page.df().drop("Type", axis=1).to_dict().get("Value")
        tmp_df = pd.DataFrame([values])
        df = pd.concat([df, tmp_df])

    # Rename columns
    to_rename = {}
    for i, v in enumerate(columns):
        to_rename[i] = v

    df = df.rename(columns=to_rename)
    
    # Filter on active notebooks
    df = df[df["Active"].astype(str) == "True"]
    print("Current notebooks:", len(df))
    return df

df_notion = get_notion_df(notion_database)
df_notion

📊 Pages in Notion DB: 572
Current notebooks: 292


,Type,Github url,No of libraries,Check logo,Lines of code,Check tags,Last edited time,Author,Check Output,Active,...,Libraries,Domain,Source,Notebook status,Cells of markdown,Check Input,Check download,Destination,Created date,Name
0,Snippet,https://github.com/jupyter-naas/awesome-notebo...,2,True,10,True,2022-03-17T23:06:00.000Z,Maxime Jublou,True,True,...,"notion, Link",⚙️ Operations,Notion,🟢,13,True,True,Notion,2022-03-17T14:25:00.000+01:00,Notion_Add_paragraph_with_link_in_page.ipynb
0,"Automation, Analytics, AI",https://github.com/jupyter-naas/awesome-notebo...,7,True,279,True,2022-03-17T23:06:00.000Z,Florent Ravenel,True,True,...,"naas, linkedin, notion, emailbuilder, gsheet, ...",🖼 Content,LinkedIn,🟢,20,True,True,Notion,2022-03-17T14:25:00.000+01:00,LinkedIn_Update_metrics_from_posts_in_Notion_c...
0,Automation,https://github.com/jupyter-naas/awesome-notebo...,3,True,29,True,2022-03-17T23:06:00.000Z,Florent Ravenel,True,True,...,"linkedin, gsheet, naas, pandas",🖼 Content,LinkedIn,🟢,14,True,True,Google Sheets,2022-03-17T14:25:00.000+01:00,LinkedIn_Send_posts_feed_to_gsheet.ipynb
0,Automation,https://github.com/jupyter-naas/awesome-notebo...,5,True,38,True,2022-03-17T23:06:00.000Z,Florent Ravenel,True,True,...,"linkedin, gsheet, random, time, pandas, datetime",🖼 Content,LinkedIn,🟢,19,True,True,Google Sheets,2022-03-17T14:25:00.000+01:00,LinkedIn_Send_likes_from_post_to_gsheet.ipynb
0,Automation,https://github.com/jupyter-naas/awesome-notebo...,3,True,38,True,2022-03-17T23:06:00.000Z,Florent Ravenel,True,True,...,"linkedin, gsheet, naas, pandas",🖼 Content,LinkedIn,🟢,14,True,True,Google Sheets,2022-03-17T14:25:00.000+01:00,LinkedIn_Send_connections_from_network_to_gshe...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,Snippet,https://github.com/jupyter-naas/awesome-notebo...,4,True,58,True,2022-03-17T23:12:00.000Z,Sanjeet Attili,True,True,...,"pandas, plotly.express, github, naas",⚙️ Operations,Github,🟢,12,True,True,Dataframe,2022-02-02T17:02:00.000+01:00,Github_Get_commits_ranking_from_repository.ipynb
0,Analytics,https://github.com/jupyter-naas/awesome-notebo...,7,True,69,True,2022-03-17T23:12:00.000Z,Sanjeet Attili,True,True,...,"pandas, requests, os, urlencode, datetime, plo...",⚙️ Operations,Github,🟢,12,True,True,Dataframe,2022-02-04T14:55:00.000+01:00,Github_Get_stargazers_from_repository.ipynb
0,Snippet,https://github.com/jupyter-naas/awesome-notebo...,4,True,27,True,2022-03-17T23:12:00.000Z,Sanjeet Attili,True,True,...,"pandas, requests, os, urlencode",⚙️ Operations,Github,🟢,12,True,True,Dataframe,2022-02-04T14:55:00.000+01:00,Github_Get_profile_from_user.ipynb
0,Snippet,https://github.com/jupyter-naas/awesome-notebo...,1,True,10,True,2022-03-18T07:55:00.000Z,Maxime Jublou,True,True,...,notion,⚙️ Operations,Notion,🟢,11,True,True,,2022-02-07T17:35:00.000+01:00,Notion_Update_pages_from_database.ipynb


### Update notebooks tags

In [24]:
def add_notebook_tags(cell, notebook_path):
    new_tags = [f"awesome-notebooks/{notebook_path}"]
    tags = cell.get('metadata').get('tags')
    if len(tags) == 0:
        tags = new_tags
    else:
        tags += new_tags
    cell["metadata"]["tags"] = tags
    return cell

### Update business tags

In [29]:
def add_business_tags(df_notion, notebook_path, cell):
    nb_name = notebook_path.split("/")[-1]
    tmp_df = df_notion[df_notion.Name == nb_name].reset_index(drop=True)
    nb_source = notebook_path.split("/")[0].lower().replace(" ", "").strip()
    nb_domain = tmp_df.loc[0, "Domain"][2:].lower().replace(" ", "").strip()
    nb_type = tmp_df.loc[0, "Type"].lower().replace(" ", "").strip()
    nb_destination = tmp_df.loc[0, "Destination"].lower().replace(" ", "").strip()
    print(nb_source, nb_domain, nb_type, nb_destination)
    return tmp_df

### Update notebook

In [30]:
def update_notebook(notebook_path):
    with open(notebook_path) as f:
        nb = json.load(f)
        
    new_cells = []
    cells = nb.get("cells")
    
    # Apply change
    for cell in cells:
        cell_type = cell.get('cell_type')
        sources = cell.get('source') 
            
        # Add new notebook tags in cell
#         cell = add_notebook_tags(cell, notebook_path)
        
        # Update business tags
        for source in sources:
            if "**Tags:**" in source:
                cell = add_business_tags(df_notion, notebook_path, cell)
                return cell
        
        new_cells.append(cell)
        
#     # Save notebook
#     nb_new = nb.copy()
#     nb_new["cells"] = new_cells
#     with open(notebook_path, 'w') as f:
#         json.dump(nb_new, f)
#     print(f"✔️ {notebook_path} saved in Naas.")
    
notebook_path = "AWS/AWS_Upload_file_to_S3_bucket.ipynb"
update_notebook(notebook_path)

aws operations snippet aws


,Type,Github url,No of libraries,Check logo,Lines of code,Check tags,Last edited time,Author,Check Output,Active,...,Libraries,Domain,Source,Notebook status,Cells of markdown,Check Input,Check download,Destination,Created date,Name
0,Snippet,https://github.com/jupyter-naas/awesome-notebo...,2,True,14,True,2022-03-17T23:11:00.000Z,Maxime Jublou,True,True,...,"boto3, boto3",⚙️ Operations,AWS,🟢,11,True,True,AWS,2021-07-14T06:30:00.000+02:00,AWS_Upload_file_to_S3_bucket.ipynb


## Output

### Update all awesome notebooks

In [56]:
# Loop on awesome-notebook dir
directories = sorted(os.listdir())
for directory in directories:
    if "." not in directory and directory != "LICENSE":
        subdirs = os.listdir(directory)
        for subdir in subdirs:
            if subdir.endswith(".ipynb"):
                notebook_path = os.path.join(directory, subdir)
                print("Started notebook... ", notebook_path)
                update_notebook(notebook_path)

Started notebook...  AWS/AWS_Upload_file_to_S3_bucket.ipynb
✔️ AWS/AWS_Upload_file_to_S3_bucket.ipynb saved in Naas.
Started notebook...  AWS/AWS_Daily_biling_notification_to_slack.ipynb
✔️ AWS/AWS_Daily_biling_notification_to_slack.ipynb saved in Naas.
Started notebook...  AWS/AWS_Get_files_from_S3_bucket.ipynb
✔️ AWS/AWS_Get_files_from_S3_bucket.ipynb saved in Naas.
